In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.optim import Adam
from torch.utils.data import TensorDataset, DataLoader

import lightning as L

In [3]:
# create vocabulary
token_to_id = {
    "who": 0,
    "is": 1,
    "charley": 2,
    "an": 3,
    "awesome": 4,
    "guy": 5,
    "<EOS>": 6,
}

# create reverse
id_to_token = dict(map(reversed, token_to_id.items()))

In [ ]:
# create inputs
inputs = torch.tensor([
    [token_to_id["who"],
     token_to_id["is"],
     token_to_id["charley"],
     token_to_id["<EOS>"],
     token_to_id["an"],
     token_to_id["awesome"],
     token_to_id["guy"]],

    [token_to_id["charley"],
     token_to_id["is"],
     token_to_id["who"],
     token_to_id["<EOS>"],
     token_to_id["an"],
     token_to_id["awesome"],
     token_to_id["guy"]]
])

# create labels
labels = torch.tensor([
    [token_to_id["is"],
     token_to_id["charley"],
     token_to_id["<EOS>"],
     token_to_id["an"],
     token_to_id["awesome"],
     token_to_id["guy"],
     token_to_id["<EOS>"]],

    [token_to_id["is"],
     token_to_id["who"],
     token_to_id["<EOS>"],
     token_to_id["an"],
     token_to_id["awesome"],
     token_to_id["guy"],
     token_to_id["<EOS>"]]
])

# create dataset from inputs and labels
dataset = TensorDataset(inputs, labels)
dataloader = DataLoader(dataset)

In [5]:
# word embedding
embedding = nn.Embedding()

TypeError: Embedding.__init__() missing 2 required positional arguments: 'num_embeddings' and 'embedding_dim'

In [6]:
# position encoding
class PositionEncoding(nn.Module):

    def __init__(self, d_model=2, max_len=6):

        super().__init__()

        pe = torch.zeros(max_len, d_model)

        position = torch.arange(start=0, end=max_len, step=1).float().unsqueeze(1)
        embedding_index = torch.arange(start=0, end=d_model, step=2).float()

        div_term = 1/torch.tensor(10000.0) ** (embedding_index / d_model)

        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)

        self.register_buffer('pe', pe)

    def forward(self, word_embeddings):

        return word_embeddings + self.pe[:word_embeddings.size(0), :]